In [ ]:
!pip install yfinance
!pip install bs4
!pip install nbformat

In [ ]:
!pip install lxml

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance Extract stock data

In [ ]:
tesla_data = yf.Ticker('TSLA')

In [ ]:
tesla_data = tesla.history(period='max')

In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

##  Use Webscraping to Extract Tesla Revenue Data

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# Step 1: Send a GET request to the webpage
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"  # Replace with the actual URL of the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code != 200:
    print("Failed to retrieve the webpage. Status code: ", response.status_code)
    exit()

# Step 2: Parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Find all tables on the webpage
tables = soup.find_all('table')

# Initialize the DataFrame
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Step 4: Loop through each table to find the relevant one
for table in tables:
    # Check if the table contains the text "Tesla Quarterly Revenue"
    if "Tesla Quarterly Revenue" in table.get_text():
        # Step 5: Loop through each row in the relevant table
        rows = table.find_all('tr')
        for row in rows[1:]:  # Skipping the header row
            columns = row.find_all('td')
            if len(columns) >= 2:  # Ensure the row has at least two columns
                date = columns[0].get_text().strip()
                revenue = columns[1].get_text().strip().replace('$', '').replace(',', '')
                # Step 6: Add the row to the DataFrame
                tesla_revenue = tesla_revenue._append({"Date": date, "Revenue": revenue}, ignore_index=True)

# Display the resulting DataFrame
# tesla_revenue.head()

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\\$',"", regex=True)

In [ ]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.head()

## Plot Tesla Stock Graph

In [ ]:
Tesla_stock_graph = make_graph(tesla_data, tesla_revenue, 'Tesla')